In [1]:
import numpy as np
import pandas as pd
from mat4py import loadmat
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn import metrics
from sklearn.utils import resample
from scipy import signal
import os
from scipy.linalg import circulant
import matplotlib.pyplot as plt
import random
import scipy
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import sklearn
from random import shuffle

In [2]:
file = open('D:\PhD topics\Datasets\Afib_datasets\store_no_noise_one_mean_final.pkl', 'rb')
#file= open('store_no_noise.pkl','rb')
obj = pickle.load(file)
file.close()

In [3]:
obj

{'ecg_train': array([[ 0.08608023,  0.08581871,  0.08469403, ...,  0.05728371,
          0.05974232,  0.06223159],
        [ 0.06668394,  0.06740808,  0.06773192, ...,  0.07126522,
          0.07099152,  0.07027662],
        [ 0.07094166,  0.07212875,  0.07197245, ...,  0.06147816,
          0.06396077,  0.06554425],
        ...,
        [-0.15816909, -0.17019585, -0.10206373, ..., -0.03658568,
         -0.04920873, -0.10357655],
        [-0.03617069, -0.02199404,  0.00623482, ..., -0.04964932,
         -0.03607034, -0.03592465],
        [-0.1316477 , -0.10271109, -0.11659159, ...,  0.01697524,
          0.07166445,  0.15376595]]),
 'ppg_train': array([[ 0.06185318,  0.06131563,  0.06131563, ...,  0.06239073,
          0.06239073,  0.06239073],
        [ 0.05970299,  0.05970299,  0.05970299, ...,  0.06024053,
          0.06024053,  0.05970299],
        [ 0.05378996,  0.05378996,  0.05325241, ...,  0.06077808,
          0.06131563,  0.06131563],
        ...,
        [ 0.79320376,  0.736

In [4]:
ppg_train=obj['ppg_train']
ppg_valid=obj['ppg_valid']
ppg_test=obj['ppg_test']
y_train=obj['y_train']
y_valid=obj['y_valid']
y_test=obj['y_test']

In [5]:
ppg_sqi_train=obj['ppg_sqi_train']
ppg_sqi_valid=obj['ppg_sqi_valid']
ppg_sqi_test=obj['ppg_sqi_test']

In [6]:
del obj

In [7]:
ppg_train

array([[ 0.06185318,  0.06131563,  0.06131563, ...,  0.06239073,
         0.06239073,  0.06239073],
       [ 0.05970299,  0.05970299,  0.05970299, ...,  0.06024053,
         0.06024053,  0.05970299],
       [ 0.05378996,  0.05378996,  0.05325241, ...,  0.06077808,
         0.06131563,  0.06131563],
       ...,
       [ 0.79320376,  0.73671454,  0.67215543, ..., -0.53686063,
        -0.49577756, -0.43195208],
       [-0.1949908 , -0.19792531, -0.1920563 , ...,  0.62080159,
         0.57531676,  0.52763105],
       [ 0.29580516,  0.34422449,  0.39337745, ..., -0.26028354,
        -0.25001277, -0.24561101]])

In [8]:
ppg_sqi_train_mean=np.mean(ppg_sqi_train, axis=0)
ppg_sqi_train_std=np.std(ppg_sqi_train, axis=0)

for i in range(ppg_sqi_train.shape[0]):
    ppg_sqi_train[i,:]=(ppg_sqi_train[i,:]-ppg_sqi_train_mean)/ppg_sqi_train_std
    
for i in range(ppg_sqi_valid.shape[0]):
    ppg_sqi_valid[i,:]=(ppg_sqi_valid[i,:]-ppg_sqi_train_mean)/ppg_sqi_train_std
    
for i in range(ppg_sqi_test.shape[0]):
    ppg_sqi_test[i,:]=(ppg_sqi_test[i,:]-ppg_sqi_train_mean)/ppg_sqi_train_std


In [9]:
ppg_train=np.expand_dims(ppg_train, axis=1)
ppg_valid=np.expand_dims(ppg_valid, axis=1)

ppg_sqi_train=np.expand_dims(ppg_sqi_train, axis=1)
ppg_sqi_valid=np.expand_dims(ppg_sqi_valid, axis=1)

In [10]:
ppg_test=np.expand_dims(ppg_test, axis=1)

ppg_sqi_test=np.expand_dims(ppg_sqi_test, axis=1)

In [11]:
num_classes=2
def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    return np.eye(num_classes, dtype='uint8')[y]

In [12]:
y_train_encoder = sklearn.preprocessing.LabelEncoder()
y_train_num = y_train_encoder.fit_transform(y_train)
y_train_wide = to_categorical(y_train_num, num_classes)

y_valid_num = y_train_encoder.fit_transform(y_valid)
y_valid_wide = to_categorical(y_valid_num, num_classes)

y_test_num = y_train_encoder.fit_transform(y_test)
y_test_wide = to_categorical(y_test_num, num_classes)

D:\conda_envs\torch_19\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [13]:
ppg_train = torch.from_numpy(ppg_train)
ppg_valid=torch.from_numpy(ppg_valid)
ppg_test = torch.from_numpy(ppg_test)

ppg_sqi_train = torch.from_numpy(ppg_sqi_train)
ppg_sqi_valid=torch.from_numpy(ppg_sqi_valid)
ppg_sqi_test = torch.from_numpy(ppg_sqi_test)

y_train_wide = torch.from_numpy(y_train_wide)
y_valid_wide = torch.from_numpy(y_valid_wide)
y_test_wide = torch.from_numpy(y_test_wide)
y_train_num = torch.from_numpy(y_train_num)
y_valid_num = torch.from_numpy(y_valid_num)
y_test_num = torch.from_numpy(y_test_num)

## PPG model

In [14]:
class ppg_model(nn.Module):
    def __init__(self):
        super().__init__()
        
        
        self.bn1_ppg=nn.BatchNorm1d(1)
        self.conv1_ppg = nn.Conv1d(in_channels=1, out_channels=5, kernel_size=30, stride=2)
        self.relu1_ppg = nn.LeakyReLU()
        self.max1_ppg = nn.MaxPool1d(2)
        self.drop1_ppg=nn.Dropout(0.25)
        self.conv2_ppg = nn.Conv1d(in_channels=5, out_channels=5, kernel_size=25, stride=1)
        self.max2_ppg = nn.MaxPool1d(2)
        self.relu2_ppg = nn.LeakyReLU()
        self.drop2_ppg=nn.Dropout(0.25)
        self.conv3_ppg = nn.Conv1d(in_channels=5, out_channels=5, kernel_size=20, stride=1)
        self.drop3_ppg=nn.Dropout(0.25)
        self.conv4_ppg = nn.Conv1d(in_channels=5, out_channels=5, kernel_size=15, stride=1)
        self.max4_ppg = nn.MaxPool1d(2)
        self.relu4_ppg = nn.LeakyReLU()
        self.bn2_ppg=nn.BatchNorm1d(5)
        self.flat_ppg=nn.Flatten()
        self.drop4_ppg=nn.Dropout(0.25)
        self.linear1_ppg=nn.Linear(660,2)
        self.softm_ppg=nn.Softmax()
        
        
        
        
        
        
    
        
    def forward(self, ppg):
        
       
        out_ppg_bn1=self.bn1_ppg(ppg)
        out_ppg_conv1= self.conv1_ppg(out_ppg_bn1)
        out_ppg_relu1 = self.relu1_ppg(out_ppg_conv1)
        out_ppg_max1=self.max1_ppg(out_ppg_relu1)
        out_ppg_drop1=self.drop1_ppg(out_ppg_max1)
        out_ppg_conv2=self.conv2_ppg(out_ppg_drop1)
        out_ppg_max2=self.max2_ppg(out_ppg_conv2)
        out_ppg_relu2=self.relu2_ppg(out_ppg_max2)
        out_ppg_drop2=self.drop2_ppg(out_ppg_relu2)
        out_ppg_conv3=self.conv3_ppg(out_ppg_drop2)
        out_ppg_drop3=self.drop3_ppg(out_ppg_conv3)
        out_ppg_conv4=self.conv4_ppg(out_ppg_drop3)
        out_ppg_max4=self.max4_ppg(out_ppg_conv4)
        out_ppg_relu4=self.relu4_ppg(out_ppg_max4)
        out_ppg_bn2=self.bn2_ppg(out_ppg_relu4)
        out_ppg_flat=self.flat_ppg(out_ppg_bn2)
        out_ppg_drop4=self.drop4_ppg(out_ppg_flat)
        out_ppg_linear=self.linear1_ppg(out_ppg_drop4)
        out_ppg_softm=self.softm_ppg(out_ppg_linear)
        


        return out_ppg_softm

In [15]:
model_ppg=ppg_model()

In [16]:
if torch.cuda.is_available():
    model_ppg = model_ppg.cuda()

In [17]:
# Define the loss function and optimizer
#loss_function = nn.functional.binary_cross_entropy
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_ppg.parameters(), lr=1.5e-5)
batch_size=32
n_batches=np.ceil(len(ppg_train)/batch_size).astype(np.int64)
valid_batch_size=100
valid_batches=np.ceil(len(ppg_valid)/valid_batch_size).astype(np.int64) 

min_valid_loss = np.inf

# Run the training loop
for epoch in range(0, 25): # 5 epochs at maximum
    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0
    current_tot_loss=0.0
    correct=0
    i=0
    # Iterate over the DataLoader for training data
    ind_list = list(range(len(ppg_train)))
    shuffle(ind_list)
    #train_new  = train[ind_list, :,:,:]
    #target_new = target[ind_list,]
    while i<n_batches:

          # Get inputs
        #local_X, local_y = X_train[i*batch_size:(i+1)*batch_size,], y_train_wide[i*batch_size:(i+1)*batch_size,]
        #local_X, local_y = X_train[i*batch_size:(i+1)*batch_size,], y_train_num[i*batch_size:(i+1)*batch_size,]
        local_ppg, local_y,  = ppg_train[ind_list[i*batch_size:(i+1)*batch_size],].cuda(),y_train_num[ind_list[i*batch_size:(i+1)*batch_size],].cuda()
        #if torch.cuda.is_available():
        #    local_X, local_y = local_X.cuda(), local_y.cuda()

          # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        outputs_ppg =model_ppg(local_ppg.float())
        
        

        # Compute loss
       
        loss = loss_function(outputs_ppg, local_y)
        
        #loss=loss1
        #print(loss)
        
        #loss = loss_function(outputs, local_y.float())
        l2_lambda = 0.01
        l2_reg = torch.tensor(0.).cuda()
        for param in model_ppg.parameters():
            l2_reg += torch.norm(param)
        loss += l2_lambda * l2_reg
        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()
        
        output = outputs_ppg.clone()
        output= torch.argmax(output, axis=1)
        
        correct += (output == local_y).float().sum()

        # Print statistics
        
        #current_loss += loss.item()
        current_tot_loss+=loss.item()
       # if i % 20 == 19:
       #     print('Loss after mini-batch %5d: %.3f' %
       #         (i + 1, current_loss / 20))
       #     current_loss = 0.0
        i=i+1
        
    
    
    correct_valid=0
    current_valid_loss = 0.0
    current_valid_tot_loss=0.0
    j=0
    while j<valid_batches:
        model_ppg.eval()     # Optional when not using Model Specific layer
        local_ppg_valid, local_y_valid = ppg_valid[j*valid_batch_size:(j+1)*valid_batch_size,].cuda(),y_valid_num[j*valid_batch_size:(j+1)*valid_batch_size,].cuda()
        target = model_ppg(local_ppg_valid.float())
        loss1 = loss_function(target,local_y_valid)
        
        loss=loss1
        
        current_valid_loss+=loss.item()
        #valid_loss = loss.item() 
        target= torch.argmax(target, axis=1)
        correct_valid += (target == local_y_valid).float().sum()
        j=j+1
    
    #print(correct)
    #print(current_tot_loss)
    accuracy = 100 * correct / len(ppg_train)
    print(f'Epoch {epoch+1} \t\t Training Loss: {current_tot_loss / n_batches} \t \t Accuracy = {accuracy}, \t Validation Loss: {current_valid_loss/ valid_batches}, \t Validation accuracy: {100*correct_valid/len(ppg_valid)}')
    
    if min_valid_loss > (current_valid_loss/valid_batches):
        print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{current_valid_loss/valid_batches:.6f}) \t Saving The Model')
        min_valid_loss = (current_valid_loss/valid_batches)
        # Saving State Dict
        torch.save(model_ppg.state_dict(), 'saved_model_ppg_no_noise_0.01.pth')
        
    
# Process is complete.
print('Training process has finished.')

Starting epoch 1


D:\conda_envs\torch_19\lib\site-packages\torch\nn\functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)
D:\conda_envs\torch_19\lib\site-packages\ipykernel_launcher.py:54: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 1 		 Training Loss: 0.6514087572371713 	 	 Accuracy = 75.78460693359375, 	 Validation Loss: 0.5799104850194607, 	 Validation accuracy: 71.11724853515625
Validation Loss Decreased(inf--->0.579910) 	 Saving The Model
Starting epoch 2
Epoch 2 		 Training Loss: 0.5734046838225278 	 	 Accuracy = 84.70208740234375, 	 Validation Loss: 0.46896425830690486, 	 Validation accuracy: 84.83905029296875
Validation Loss Decreased(0.579910--->0.468964) 	 Saving The Model
Starting epoch 3
Epoch 3 		 Training Loss: 0.5592253665908947 	 	 Accuracy = 84.92825317382812, 	 Validation Loss: 0.465575939730594, 	 Validation accuracy: 84.86842346191406
Validation Loss Decreased(0.468964--->0.465576) 	 Saving The Model
Starting epoch 4
Epoch 4 		 Training Loss: 0.5519083332396427 	 	 Accuracy = 85.08907318115234, 	 Validation Loss: 0.45845567238958257, 	 Validation accuracy: 85.68492126464844
Validation Loss Decreased(0.465576--->0.458456) 	 Saving The Model
Starting epoch 5
Epoch 5 		 Training Loss: 0.5398

In [19]:
model_saved = ppg_model()
model_saved.load_state_dict(torch.load('saved_model_ppg_no_noise_0.01.pth'))
model_saved.eval()
model_saved=model_saved.cpu()

In [21]:
outputs_test = model_saved(ppg_test.float())
outputs_test = outputs_test.clone()
outputs_test= torch.argmax(outputs_test, axis=1)

print("PPG output")
        
correct = (outputs_test == y_test_num).float().sum()
print(correct/len(ppg_test))

cm=confusion_matrix(y_test_num.cpu(), outputs_test.cpu())
print(cm)



D:\conda_envs\torch_19\lib\site-packages\ipykernel_launcher.py:54: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


PPG output
tensor(0.8671)
[[6705 1806]
 [  49 5399]]
